In [41]:
import configparser
import json
import requests
import sys
import csv

GRANT_TYPE = '/services/oauth2/token?grant_type=password'
ACCESS_TOKEN = 'access_token'
OAUTH_HEADER = {'Authorization': ''}
INSTANCE_URL = 'instance_url'
API_INSTANCE = '/services/data/v43.0/'
OBJECT_INST = "sobjects/"
QUERY_INST = 'query/?q='
CLIENT_KEY = 'YjkyMzJmNDUzMGNmOTYzODVlNDQ0ZjE2OTUxOTU1NDI5MGViN2I3NWU3YTFjNmY300D1k0000004bBG'
CLIENT_SECRET = '7017597587192369305'
USERNAME = 'bedantika.lahiri@omnicell.com.fulldev'
PASSWORD = 'Welcome1qEGoPJmPYuLrJY0syfKg0ivDN'
SERVER_URL = 'https://test.salesforce.com'


next_record_url = None
object_dir = None
record_size = 0
total_size = None

def acquire_sfdc_acces():
    try:
        login_url = SERVER_URL + GRANT_TYPE
        print(login_url)
        param = {'client_id': CLIENT_KEY, 'client_secret': CLIENT_SECRET, 'username': USERNAME, 'password': PASSWORD}
        print(param)
        response = requests.post(login_url, data = param)
        print(response.status_code, response.reason)
        global ACCESS_TOKEN
        global INSTANCE_URL

        if response.status_code == 200:
            json_data = json.loads(response.text)
            ACCESS_TOKEN = json_data[ACCESS_TOKEN]
            INSTANCE_URL = json_data[INSTANCE_URL]
            print('Access Token: ##' + ACCESS_TOKEN[10:-10] + '##')
            print('Instance Url: ' + INSTANCE_URL[18:-4])
            OAUTH_HEADER['Authorization'] = 'OAuth ' + ACCESS_TOKEN
    except Exception as ex:
        print(str(ex))
        

def fetch_query_data(query, is_next_url):
    
    
    global record_size
    global next_record_url
    global total_size
    login_url = None
    
    
    if is_next_url:
        login_url = INSTANCE_URL + query
    else:
        login_url = INSTANCE_URL + API_INSTANCE + QUERY_INST + query
        
    response = requests.get(login_url, headers = OAUTH_HEADER)
    
    raw_response = None
    
    if response.status_code == 200:
        raw_response = json.loads(response.text)
        
    records = raw_response['records']

    total_size = raw_response['totalSize']
    
    print('Records Fetched: ' + str(len(records)))
    
    record_size += len(records)
    
    if 'nextRecordsUrl' in raw_response and raw_response['nextRecordsUrl'] != None:
        
        next_record_url = raw_response['nextRecordsUrl']
        
        print('Next batch url: ' + str(next_record_url))
    return records


acquire_sfdc_acces()            


qlfields = ['Quote__r.SAP_Quote_Number_Truncated__c','Conga_Product1__c','Conga_Quantity__c','Conga_List_Price1__c','Conga_Unit_Price_product__c','Conga_Extended_Price_Materials__c',
'Conga_Unit_Services__c','Conga_Extended_Services__c','Conga1_USA_List_Price__c']
            
str1 = ','.join(qlfields)
where = 'WHERE+Quote__r.SAP_Quote_Number_Truncated__c = \'5370615\''    
zsoql = "SELECT+" + str1 + "+FROM Quote_Line__c+" + where
    
print(zsoql)

login_url = INSTANCE_URL + API_INSTANCE + QUERY_INST + zsoql

response = requests.get(login_url, headers = OAUTH_HEADER)

print(response)

raw_response = None
    
if response.status_code == 200:
    raw_response = json.loads(response.text)
    

#for result in raw_response['records']:
#    record[u'SAP_Quote_Number_Truncated__c']=result[u'attributes']
#    result[u'lng']=result[u'location'][u'lng']
 #   del result[u'attributes']
#    print(record)

    
           
QUOTATION_ID,PRODUCT,QTY,CONTRACT_LIST_PRICE ,UNIT_PRICE= [],[],[],[],[]
EXTEND_PRICE,UNIT_SERVICE,EXTEND_SERVICE,USA_LIST_PRICE = [],[],[],[]

for result in raw_response['records']:
    QUOTATION_ID.append(result[u'Quote__r'][u'SAP_Quote_Number_Truncated__c'])
    PRODUCT.append(result[u'Conga_Product1__c'])
    QTY.append(result[u'Conga_Quantity__c'])
    CONTRACT_LIST_PRICE.append(result[u'Conga_List_Price1__c'])
    UNIT_PRICE.append(result[u'Conga_Unit_Price_product__c'])
    EXTEND_PRICE.append(result[u'Conga_Extended_Price_Materials__c'])
    UNIT_SERVICE.append(result[u'Conga_Unit_Services__c'])
    EXTEND_SERVICE.append(result[u'Conga_Extended_Services__c'])
    USA_LIST_PRICE.append(result[u'Conga1_USA_List_Price__c'])
    
   # lng.append(result[u'location'][u'lng'])
 #   el.append(result[u'elevation'])
df3 = pd.DataFrame([QUOTATION_ID,PRODUCT,CONTRACT_LIST_PRICE,EXTEND_PRICE,EXTEND_SERVICE,QTY,UNIT_PRICE,UNIT_SERVICE,USA_LIST_PRICE]).T

zcols = ['QUOTATION_ID', 'PRODUCT', 'CONTRACT_LIST_PRICE', 'EXTEND_PRICE', 'EXTEND_SERVICE',  'QTY', 'UNIT_PRICE', 'UNIT_SERVICE','USA_LIST_PRICE']
df3.columns = zcols

print(df3)

df3.set_index(["QUOTATION_ID", "PRODUCT"], inplace = True, 
                            append = True, drop = True) 

https://test.salesforce.com/services/oauth2/token?grant_type=password
{'client_id': 'YjkyMzJmNDUzMGNmOTYzODVlNDQ0ZjE2OTUxOTU1NDI5MGViN2I3NWU3YTFjNmY300D1k0000004bBG', 'client_secret': '7017597587192369305', 'username': 'bedantika.lahiri@omnicell.com.fulldev', 'password': 'Welcome1qEGoPJmPYuLrJY0syfKg0ivDN'}
200 OK
Access Token: ##04bBG!AQkAQEdhd0KVZZtNj_amV0ARLJEJhXJi5ZST4iIde6k9T8tqtUSFI8AVHOnEhE9dlvqSoTFMGW0ytP3Qfk5EJR##
Instance Url: fulldev.cs78.my.salesforce
SELECT+Quote__r.SAP_Quote_Number_Truncated__c,Conga_Product1__c,Conga_Quantity__c,Conga_List_Price1__c,Conga_Unit_Price_product__c,Conga_Extended_Price_Materials__c,Conga_Unit_Services__c,Conga_Extended_Services__c,Conga1_USA_List_Price__c+FROM Quote_Line__c+WHERE+Quote__r.SAP_Quote_Number_Truncated__c = '5370615'
<Response [200]>
   QUOTATION_ID         PRODUCT CONTRACT_LIST_PRICE EXTEND_PRICE  \
0       5370615         FREIGHT                   0      1692.45   
1       5370615     MED-FRM-101              234.69       421.2

In [10]:
records = raw_response['records']

print(records)
total_size = raw_response['totalSize']
print('Records Fetched: ' + str(len(records)))
record_size += len(records)

print(qlfields)

for obj in records:
    data = []
    for field in qlfields:
        if field in obj :
        
        #and obj[field] != None:
            data.append(obj[field])
            
    print(data)         

[{'attributes': {'type': 'Quote_Line__c', 'url': '/services/data/v43.0/sobjects/Quote_Line__c/a301k0000000cCYAAY'}, 'Quote__r': {'attributes': {'type': 'Quote', 'url': '/services/data/v43.0/sobjects/Quote/0Q01k0000000aTICAY'}, 'SAP_Quote_Number_Truncated__c': '5370615'}, 'Conga_Product1__c': 'FREIGHT', 'Conga_Quantity__c': 1.0, 'Conga_List_Price__c': 0.0, 'Conga_Unit_Price__c': 1692.45, 'Conga_Extended_Price_Materials__c': 1692.45, 'Conga_Unit_Services__c': 0.0, 'Conga_Extended_Services__c': 0.0, 'Conga1_USA_List_Price__c': 0.0}, {'attributes': {'type': 'Quote_Line__c', 'url': '/services/data/v43.0/sobjects/Quote_Line__c/a301k0000000cCZAAY'}, 'Quote__r': {'attributes': {'type': 'Quote', 'url': '/services/data/v43.0/sobjects/Quote/0Q01k0000000aTICAY'}, 'SAP_Quote_Number_Truncated__c': '5370615'}, 'Conga_Product1__c': 'MED-FRM-101', 'Conga_Quantity__c': 1.0, 'Conga_List_Price__c': 421.23, 'Conga_Unit_Price__c': 421.23, 'Conga_Extended_Price_Materials__c': 421.23, 'Conga_Unit_Services__c'

NameError: name 'attributes' is not defined

In [13]:
import pandas as apd

type(records)

list

In [7]:
qlfields = ['id','Conga_Product1__c','Conga_Quantity__c','Conga_List_Price__c','Conga_Unit_Price__c','Conga_Extended_Price_Materials__c',
'Conga_Unit_Services__c','Conga_Extended_Services__c','Conga1_USA_List_Price__c']
            
qlfields

['id',
 'Conga_Product1__c',
 'Conga_Quantity__c',
 'Conga_List_Price__c',
 'Conga_Unit_Price__c',
 'Conga_Extended_Price_Materials__c',
 'Conga_Unit_Services__c',
 'Conga_Extended_Services__c',
 'Conga1_USA_List_Price__c']

In [19]:
str1 = ','.join(qlfields)
where = 'WHERE+Quote__r.SAP_Quote_Number_Truncated__c = \'5370615\''

In [12]:
str1,

'id,Conga_Product1__c,Conga_Quantity__c,Conga_List_Price__c,Conga_Unit_Price__c,Conga_Extended_Price_Materials__c,Conga_Unit_Services__c,Conga_Extended_Services__c,Conga1_USA_List_Price__c'

In [16]:
zsoql = "SELECT" + str1 + "+FROM Quote_Line__c+"

In [20]:
zsoql+where

"SELECTid,Conga_Product1__c,Conga_Quantity__c,Conga_List_Price__c,Conga_Unit_Price__c,Conga_Extended_Price_Materials__c,Conga_Unit_Services__c,Conga_Extended_Services__c,Conga1_USA_List_Price__c+FROM+WHERE+Quote__r.SAP_Quote_Number_Truncated__c = '5370615'"

In [4]:
from simple_salesforce import Salesforce

ModuleNotFoundError: No module named 'api'

In [14]:
type(records)

list

In [6]:
import pandas as pd 

df = pd.DataFrame(records)

In [8]:
print(df)


    Conga1_USA_List_Price__c  Conga_Extended_Price_Materials__c  \
0                       0.00                            1692.45   
1                     699.90                             421.23   
2                     512.34                             305.81   
3                        NaN                                NaN   
4                        NaN                                NaN   
5                        NaN                                NaN   
6                        NaN                                NaN   
7                        NaN                                NaN   
8                   20000.00                             421.23   
9                   26060.00                             305.81   
10                      0.00                               0.02   

    Conga_Extended_Services__c  Conga_List_Price__c Conga_Product1__c  \
0                          0.0                 0.00           FREIGHT   
1                        105.0               421.

In [25]:
print(df.columns)


Index(['Conga1_USA_List_Price__c', 'Conga_Extended_Price_Materials__c',
       'Conga_Extended_Services__c', 'Conga_List_Price__c',
       'Conga_Product1__c', 'Conga_Quantity__c', 'Conga_Unit_Price__c',
       'Conga_Unit_Services__c', 'Id', 'attributes'],
      dtype='object')


In [31]:
type(df)

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')

In [27]:
# create a dictionary with five fields each  
data = { 
    'A':['A1', 'A2', 'A3', 'A4', 'A5'],  
    'B':['B1', 'B2', 'B3', 'B4', 'B5'],  
    'C':['C1', 'C2', 'C3', 'C4', 'C5'],  
    'D':['D1', 'D2', 'D3', 'D4', 'D5'],  
    'E':['E1', 'E2', 'E3', 'E4', 'E5'] } 
  
# Convert the dictionary into DataFrame  
df = pd.DataFrame(data) 

In [29]:
type(df)
print(df.columns)

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')


In [33]:
df.columns

Index(['Conga1_USA_List_Price__c', 'Conga_Extended_Price_Materials__c',
       'Conga_Extended_Services__c', 'Conga_List_Price__c',
       'Conga_Product1__c', 'Conga_Quantity__c', 'Conga_Unit_Price__c',
       'Conga_Unit_Services__c', 'Id', 'attributes'],
      dtype='object')

In [34]:
qlfields

['id',
 'Conga_Product1__c',
 'Conga_Quantity__c',
 'Conga_List_Price__c',
 'Conga_Unit_Price__c',
 'Conga_Extended_Price_Materials__c',
 'Conga_Unit_Services__c',
 'Conga_Extended_Services__c',
 'Conga1_USA_List_Price__c']

In [35]:
df2 = pd.DataFrame(df,columns = qlfields)

In [36]:
df2

,id,Conga_Product1__c,Conga_Quantity__c,Conga_List_Price__c,Conga_Unit_Price__c,Conga_Extended_Price_Materials__c,Conga_Unit_Services__c,Conga_Extended_Services__c,Conga1_USA_List_Price__c
0,NaN,FREIGHT,1.0,0.00,1692.45,1692.45,0.0,0.0,0.00
1,NaN,MED-FRM-101,1.0,421.23,421.23,421.23,105.0,105.0,699.90
2,NaN,MED-AUX-102,1.0,305.81,305.81,305.81,50.0,50.0,512.34
3,NaN,MED-PCB-001,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4,NaN,MED-SCN-001,0.0,NaN,NaN,NaN,NaN,0.0,NaN
5,NaN,MED-LIC-004,0.0,NaN,NaN,NaN,NaN,0.0,NaN
6,NaN,MED-LIC-005,0.0,NaN,NaN,NaN,NaN,0.0,NaN
7,NaN,MED-LIC-004-SV,0.0,NaN,NaN,NaN,NaN,0.0,NaN
8,NaN,MED-FRM-101-SV,0.0,234.69,105.00,421.23,105.0,105.0,20000.00
9,NaN,MED-AUX-102-SV,0.0,305.81,50.00,305.81,0.0,50.0,26060.00


In [39]:
print(df2)

    id Conga_Product1__c  Conga_Quantity__c  Conga_List_Price__c  \
0  NaN           FREIGHT                1.0                 0.00   
1  NaN       MED-FRM-101                1.0               421.23   
2  NaN       MED-AUX-102                1.0               305.81   
3  NaN       MED-PCB-001                0.0                  NaN   
4  NaN       MED-SCN-001                0.0                  NaN   
5  NaN       MED-LIC-004                0.0                  NaN   
6  NaN       MED-LIC-005                0.0                  NaN   
7  NaN    MED-LIC-004-SV                0.0                  NaN   
8  NaN    MED-FRM-101-SV                0.0               234.69   
9  NaN    MED-AUX-102-SV                0.0               305.81   
10 NaN           RENEWAL                2.0                 0.01   

    Conga_Unit_Price__c  Conga_Extended_Price_Materials__c  \
0               1692.45                            1692.45   
1                421.23                             421.23 